In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, WeightedRandomSampler
from torchvision import datasets, transforms
from torch.optim import AdamW
from torch.optim.lr_scheduler import OneCycleLR
from torch.cuda.amp import autocast, GradScaler
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import pickle

In [ ]:
# Configuration with optimizations
data_dir = r'C:\Users\Jatin\Desktop\oncogenesis\data\lungs_data'
batch_size = 32  # Increased from 16 for better GPU utilization
num_classes = 3  # Adenocarcinoma, Benign_Tissue, Squamous_Cell_Carcinoma
epochs = 60
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Enable cuDNN autotuner for faster convolutions
if torch.cuda.is_available():
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.enabled = True
    print(f"Using device: {device}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print(f"Using device: {device}")

print(f"Batch size: {batch_size}")

Using device: cuda
GPU: NVIDIA GeForce RTX 3050 Laptop GPU
Memory: 4.29 GB
Batch size: 32, Num workers: 4


In [3]:
# Custom Gaussian Noise augmentation
class GaussianNoise:
    def __init__(self, mean=0., std=1.):
        self.std = std
        self.mean = mean
        
    def __call__(self, tensor):
        return tensor + torch.randn(tensor.size()) * self.std + self.mean
    
    def __repr__(self):
        return self.__class__.__name__ + '(mean={0}, std={1})'.format(self.mean, self.std)

# Training transforms with extensive augmentation for histopathology images
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Higher resolution for histopathology
    transforms.RandomRotation(90),  # Histopathology images can be rotated in any direction
    transforms.RandomResizedCrop((224, 224), scale=(0.6, 1.0)),  # Scale variation
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),  # Histopathology images can be flipped
    transforms.ColorJitter(
        brightness=0.3,
        contrast=0.3,
        saturation=0.2,
        hue=0.1
    ),
    transforms.RandomAffine(
        degrees=90,
        translate=(0.15, 0.15),
        scale=(0.8, 1.2),
        shear=10
    ),
    transforms.RandomPerspective(distortion_scale=0.2, p=0.3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # ImageNet stats
    GaussianNoise(0., 0.01)  # Slight noise for robustness
])

# Validation/Test transforms (no augmentation)
val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [4]:
# Load the full dataset
full_dataset = datasets.ImageFolder(root=data_dir, transform=train_transform)

# Split the dataset into train, validation, and test sets
train_size = int(0.8 * len(full_dataset))
val_size = int(0.1 * len(full_dataset))
test_size = len(full_dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(
    full_dataset, [train_size, val_size, test_size],
    generator=torch.Generator().manual_seed(42)
)

# Calculate class weights for handling class imbalance
train_targets = [full_dataset.targets[i] for i in train_dataset.indices]
class_sample_counts = np.bincount(train_targets)
class_weights = 1. / class_sample_counts
samples_weights = [class_weights[t] for t in train_targets]

print(f"Dataset loaded: {len(full_dataset)} total images")
print(f"Train: {train_size}, Validation: {val_size}, Test: {test_size}")
print(f"Classes: {full_dataset.classes}")
print(f"Class distribution in training: {class_sample_counts}")

Dataset loaded: 15000 total images
Train: 12000, Validation: 1500, Test: 1500
Classes: ['Adenocarcinoma', 'Benign_Tissue', 'Squamous_Cell_Carcinoma']
Class distribution in training: [4010 4041 3949]


In [ ]:
# Create weighted sampler for balanced training
sampler = WeightedRandomSampler(
    weights=samples_weights, 
    num_samples=len(samples_weights), 
    replacement=True
)

# Set validation and test transforms (no augmentation)
val_dataset.dataset.transform = val_transform
test_dataset.dataset.transform = val_transform

# Create optimized data loaders (Windows-compatible)
train_loader = DataLoader(
    train_dataset, 
    batch_size=batch_size, 
    sampler=sampler,
    num_workers=0,  # Set to 0 for Windows to avoid multiprocessing issues
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset, 
    batch_size=batch_size * 2,  # Larger batch for validation (no gradients)
    shuffle=False,
    num_workers=0,
    pin_memory=True
)

test_loader = DataLoader(
    test_dataset, 
    batch_size=batch_size * 2, 
    shuffle=False,
    num_workers=0,
    pin_memory=True
)

print(f"Data loaders created with optimizations:")
print(f"Train batches: {len(train_loader)}, Val batches: {len(val_loader)}, Test batches: {len(test_loader)}")
print(f"Train batch size: {batch_size}, Val/Test batch size: {batch_size * 2}")
print("Note: num_workers=0 for Windows compatibility")

Data loaders created with optimizations:
Train batches: 375, Val batches: 24, Test batches: 24
Train batch size: 32, Val/Test batch size: 64


In [6]:
# Advanced CNN architecture for lung histopathology classification
class LungCNN(nn.Module):
    def __init__(self, num_classes=3):
        super(LungCNN, self).__init__()
        
        # First convolution block - capture basic features
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            nn.Dropout(0.2)
        )
        
        # Second convolution block - intermediate features
        self.conv2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            nn.Dropout(0.3)
        )
        
        # Third convolution block - complex features
        self.conv3 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            nn.Dropout(0.4)
        )
        
        # Fourth convolution block - high-level features
        self.conv4 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            nn.Dropout(0.5)
        )
        
        # Adaptive pooling to handle variable input sizes
        self.adaptive_pool = nn.AdaptiveAvgPool2d((4, 4))
        
        # Fully connected layers with progressive dimension reduction
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(512 * 4 * 4, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.4),
            nn.Linear(512, num_classes)
        )
        
        # Initialize weights using Kaiming initialization
        self._initialize_weights()
    
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d) or isinstance(m, nn.BatchNorm1d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.adaptive_pool(x)
        x = self.classifier(x)
        return x

# Initialize model
model = LungCNN(num_classes=num_classes).to(device)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"\nModel initialized with {trainable_params:,} trainable parameters (Total: {total_params:,})")


Model initialized with 16,559,683 trainable parameters (Total: 16,559,683)


In [7]:
# Mixup data augmentation for better generalization
def mixup_data(x, y, alpha=0.2):
    '''Returns mixed inputs, pairs of targets, and lambda'''
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    batch_size = x.size()[0]
    index = torch.randperm(batch_size).to(device)

    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

# Loss function with class weights
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float).to(device)
criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)

# Optimizer with weight decay (L2 regularization)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.01)

# Learning rate scheduler with warmup
scheduler = OneCycleLR(
    optimizer,
    max_lr=0.001,
    epochs=epochs,
    steps_per_epoch=len(train_loader),
    pct_start=0.2,  # Warmup for first 20% of training
    div_factor=25,  # LR starts at max_lr/25
    final_div_factor=1000  # Final LR is max_lr/1000
)

# Initialize gradient scaler for mixed precision training
scaler = GradScaler()
use_amp = torch.cuda.is_available()  # Use AMP only if CUDA is available

print("Training configuration:")
print(f"Optimizer: AdamW with weight decay 0.01")
print(f"Scheduler: OneCycleLR with max_lr=0.001")
print(f"Loss: CrossEntropyLoss with class weights")
print(f"Mixed Precision (AMP): {'Enabled' if use_amp else 'Disabled'}")
print(f"Class weights: {class_weights}")

Training configuration:
Optimizer: AdamW with weight decay 0.01
Scheduler: OneCycleLR with max_lr=0.001
Loss: CrossEntropyLoss with class weights
Mixed Precision (AMP): Enabled
Class weights: [0.00024938 0.00024746 0.00025323]


C:\Users\Jatin\AppData\Local\Temp\ipykernel_19248\717511980.py:38: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


In [ ]:
# Training function with progress tracking and optimizations
def plot_confusion_matrix(y_true, y_pred, classes):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10,8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
    plt.xlabel('Predicted', fontsize=12)
    plt.ylabel('True', fontsize=12)
    plt.title('Confusion Matrix - Lung Cancer Classification', fontsize=14)
    plt.tight_layout()
    plt.show()

# Training tracking
best_val_acc = 0
best_model_state = None
patience = 12
patience_counter = 0
train_losses = []
val_losses = []
train_accs = []
val_accs = []
val_predictions = []
val_targets = []

print("Starting training with optimizations...\n")

for epoch in range(epochs):
    # ==================== Training Phase ====================
    model.train()
    running_loss = 0
    correct = 0
    total = 0
    
    train_loop = tqdm(train_loader, desc=f'Epoch {epoch+1}/{epochs} [Train]')
    for batch_idx, (images, labels) in enumerate(train_loop):
        images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)
        
        # Apply mixup augmentation
        images, labels_a, labels_b, lam = mixup_data(images, labels, alpha=0.2)
        
        optimizer.zero_grad(set_to_none=True)  # Faster than zero_grad()
        
        # Mixed precision training
        with autocast(enabled=use_amp):
            outputs = model(images)
            loss = mixup_criterion(criterion, outputs, labels_a, labels_b, lam)
        
        # Backward pass with gradient scaling
        scaler.scale(loss).backward()
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()
        
        # Metrics
        running_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (lam * (predicted == labels_a).sum().item()
                   + (1 - lam) * (predicted == labels_b).sum().item())
        
        # Update progress bar less frequently for speed
        if batch_idx % 5 == 0:
            train_loop.set_postfix({
                'Loss': f'{running_loss/total:.4f}',
                'Acc': f'{correct/total:.4f}',
                'LR': f'{optimizer.param_groups[0]["lr"]:.6f}'
            })
    
    train_loss = running_loss / total
    train_acc = correct / total
    train_losses.append(train_loss)
    train_accs.append(train_acc)
    
    # ==================== Validation Phase ====================
    model.eval()
    val_loss = 0
    val_correct = 0
    val_total = 0
    val_preds = []
    val_true = []
    
    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc=f'Epoch {epoch+1}/{epochs} [Val]'):
            images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)
            
            with autocast(enabled=use_amp):
                outputs = model(images)
                loss = criterion(outputs, labels)
            
            val_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()
            
            val_preds.extend(predicted.cpu().numpy())
            val_true.extend(labels.cpu().numpy())
    
    val_loss = val_loss / val_total
    val_acc = val_correct / val_total
    val_losses.append(val_loss)
    val_accs.append(val_acc)
    
    # Save predictions for confusion matrix
    val_predictions = val_preds
    val_targets = val_true
    
    print(f"\n{'='*60}")
    print(f"Epoch {epoch+1}/{epochs} Summary:")
    print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
    print(f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")
    print(f"Learning Rate: {optimizer.param_groups[0]['lr']:.6f}")
    print(f"{'='*60}\n")
    
    # ==================== Model Saving & Early Stopping ====================
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_model_state = model.state_dict().copy()
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'val_acc': val_acc,
            'val_loss': val_loss,
            'class_names': full_dataset.classes,
        }, 'lung_cnn_best.pth')
        print(f"✓ New best model saved! Validation Accuracy: {best_val_acc:.4f}\n")
        patience_counter = 0
    else:
        patience_counter += 1
        print(f"No improvement. Patience: {patience_counter}/{patience}\n")
        if patience_counter >= patience:
            print(f"\n{'='*60}")
            print(f"Early stopping triggered after {epoch+1} epochs!")
            print(f"Best validation accuracy: {best_val_acc:.4f}")
            print(f"{'='*60}\n")
            break

# Load best model
if best_model_state is not None:
    model.load_state_dict(best_model_state)
    print(f"\n✓ Loaded best model with validation accuracy: {best_val_acc:.4f}\n")
    
    # Plot final confusion matrix
    plot_confusion_matrix(val_targets, val_predictions, full_dataset.classes)
    
    # Print classification report
    print("\nClassification Report:")
    print(classification_report(val_targets, val_predictions, target_names=full_dataset.classes))

print("\nTraining completed!")

Starting training with optimizations...



Epoch 1/60 [Train]:   0%|          | 0/375 [00:00<?, ?it/s]

In [ ]:
# Plot training history
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Loss plot
axes[0].plot(train_losses, label='Train Loss', linewidth=2, color='#2E86AB')
axes[0].plot(val_losses, label='Validation Loss', linewidth=2, color='#A23B72')
axes[0].set_title('Training and Validation Loss', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Epoch', fontsize=12)
axes[0].set_ylabel('Loss', fontsize=12)
axes[0].legend(fontsize=10)
axes[0].grid(True, alpha=0.3)

# Accuracy plot
axes[1].plot(train_accs, label='Train Accuracy', linewidth=2, color='#2E86AB')
axes[1].plot(val_accs, label='Validation Accuracy', linewidth=2, color='#A23B72')
axes[1].set_title('Training and Validation Accuracy', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Epoch', fontsize=12)
axes[1].set_ylabel('Accuracy', fontsize=12)
axes[1].legend(fontsize=10)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nFinal Results:")
print(f"Best Validation Accuracy: {best_val_acc:.4f}")
print(f"Final Train Accuracy: {train_accs[-1]:.4f}")
print(f"Final Validation Accuracy: {val_accs[-1]:.4f}")

In [ ]:
# Evaluate on test set with optimizations
print("Evaluating on test set...\n")
model.eval()
test_loss = 0
test_correct = 0
test_total = 0
test_preds = []
test_true = []

with torch.no_grad():
    for images, labels in tqdm(test_loader, desc='Testing'):
        images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)
        
        with autocast(enabled=use_amp):
            outputs = model(images)
            loss = criterion(outputs, labels)
        
        test_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs.data, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()
        
        test_preds.extend(predicted.cpu().numpy())
        test_true.extend(labels.cpu().numpy())

test_loss = test_loss / test_total
test_acc = test_correct / test_total

print(f"\n{'='*60}")
print(f"Test Set Results:")
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")
print(f"{'='*60}\n")

# Plot test confusion matrix
plot_confusion_matrix(test_true, test_preds, full_dataset.classes)

# Print detailed classification report
print("\nTest Set Classification Report:")
print(classification_report(test_true, test_preds, target_names=full_dataset.classes, digits=4))

In [ ]:
# Save the complete model and checkpoint
print("Saving models...\n")

# Save the full model
torch.save(model, "lung_cnn_full_model.pth")

# Save a comprehensive checkpoint with all necessary information
checkpoint = {
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'scheduler_state_dict': scheduler.state_dict(),
    'class_names': full_dataset.classes,
    'class_to_idx': full_dataset.class_to_idx,
    'input_size': (224, 224),  # The image size we used for training
    'normalize_mean': [0.485, 0.456, 0.406],  # ImageNet stats
    'normalize_std': [0.229, 0.224, 0.225],
    'best_val_acc': best_val_acc,
    'test_acc': test_acc,
    'num_classes': num_classes,
    'training_history': {
        'train_losses': train_losses,
        'val_losses': val_losses,
        'train_accs': train_accs,
        'val_accs': val_accs
    }
}
torch.save(checkpoint, "lung_cnn_checkpoint.pth")

# Save class names separately for easy access
with open("lung_class_names.pkl", "wb") as f:
    pickle.dump(full_dataset.classes, f)

print("=" * 60)
print("Models saved successfully!")
print("=" * 60)
print("\nSaved files:")
print("1. 'lung_cnn_full_model.pth' - Full model for simple loading")
print("2. 'lung_cnn_best.pth' - Best checkpoint during training")
print("3. 'lung_cnn_checkpoint.pth' - Complete checkpoint with all parameters")
print("4. 'lung_class_names.pkl' - Class names mapping")
print("\nModel Summary:")
print(f"Classes: {full_dataset.classes}")
print(f"Class mapping: {full_dataset.class_to_idx}")
print(f"Best Validation Accuracy: {best_val_acc:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")
print(f"Input Size: 224x224")
print("=" * 60)

In [ ]:
# Example: How to load and use the model for inference
print("Example: Loading model for inference\n")

# Method 1: Load the full model
loaded_model = torch.load("lung_cnn_full_model.pth")
loaded_model.eval()

# Method 2: Load from checkpoint (more flexible)
checkpoint_loaded = torch.load("lung_cnn_checkpoint.pth")
model_for_inference = LungCNN(num_classes=checkpoint_loaded['num_classes']).to(device)
model_for_inference.load_state_dict(checkpoint_loaded['model_state_dict'])
model_for_inference.eval()

print("✓ Model loaded successfully!")
print(f"Classes: {checkpoint_loaded['class_names']}")
print(f"Model accuracy on test set: {checkpoint_loaded['test_acc']:.4f}")

# Load class names
with open("lung_class_names.pkl", "rb") as f:
    class_names = pickle.load(f)
print(f"\nClass names loaded: {class_names}")

In [ ]:
# Visualize sample predictions
print("Visualizing sample predictions from test set...\n")

# Get a batch from test loader
dataiter = iter(test_loader)
images, labels = next(dataiter)
images, labels = images.to(device), labels.to(device)

# Make predictions
with torch.no_grad():
    outputs = model(images)
    _, predictions = torch.max(outputs, 1)
    probabilities = F.softmax(outputs, dim=1)

# Denormalize images for visualization
def denormalize(tensor, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]):
    for t, m, s in zip(tensor, mean, std):
        t.mul_(s).add_(m)
    return tensor

# Plot sample predictions
fig, axes = plt.subplots(2, 4, figsize=(16, 8))
axes = axes.ravel()

for idx in range(min(8, len(images))):
    img = images[idx].cpu().clone()
    img = denormalize(img)
    img = img.permute(1, 2, 0).numpy()
    img = np.clip(img, 0, 1)
    
    axes[idx].imshow(img)
    axes[idx].axis('off')
    
    true_label = full_dataset.classes[labels[idx]]
    pred_label = full_dataset.classes[predictions[idx]]
    confidence = probabilities[idx][predictions[idx]].item() * 100
    
    color = 'green' if predictions[idx] == labels[idx] else 'red'
    axes[idx].set_title(f'True: {true_label}\nPred: {pred_label}\nConf: {confidence:.1f}%',
                       fontsize=10, color=color, fontweight='bold')

plt.suptitle('Sample Predictions from Test Set', fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()